In [242]:
import pandas as pd
import time,datetime
import numpy as np
import glob
from pandas import *
import scipy.optimize
%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/Users/yerin/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['unique', 'plotting', 'test', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Data Preprocessing
## 1. Time series data
##### -dic_data : all data group by date
##### -sedol list : all stock lists encoded by SEDOL
## 2. Risk data
##### - risk matrix

In [99]:
#timeseries data
all_data=pd.read_csv('./data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

dic_data = {k: v for k, v in all_data.groupby('date')}

In [100]:
#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('./data/Riskmodels/cov_mat_%s.csv'%(date_str))
    #risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])

## 3. Historical data

In [102]:
sp=pd.read_csv('./data/company info.csv')

a=list(sp['name'])

f_data=dic_data[list(dic_data.keys())[0]]
for i in range(len(a)):
    a[i]=a[i][:6]
sp['name2']=a

b=list(f_data['name'])
for i in range(len(b)):
    b[i]=b[i][:6]
f_data['name2']=b

sp_=pd.merge(f_data,sp, on='name2', how='left')
#sp_.to_csv('./list.csv')
#수작업...

In [107]:
for2007=pd.read_csv('./list.csv')
aa = glob.glob('/Users/yerin/Downloads/Data/Stocks' + "/*.txt")

history_list=[0]*len(aa)
for i in range(len(aa)):
    history_list[i]=aa[i][35:aa[i].index('.')]


In [150]:
for2007['nyse']=for2007['nyse'].str.lower()
list2007=list(for2007['nyse'])

da='2006-01-03'
close=[0]*len(list2007)
#loc=[0]*len(list2007)
for i in range(len(list2007)):
    if list2007[i] in history_list:
        ddd=history_list.index(list2007[i])
        ak=pd.read_csv(aa[ddd])
        if da in list(ak['Date']):
            akk=float(ak.loc[ak['Date']==da]['Close'])
            close[i]=akk
        else:
            close[i]='NOO'
    else:
        close[i]='-'
for2007['Close']=close

for2007=for2007.loc[~for2007['Close'].isin(['NOO','-'])]

In [238]:
def load_data():
    symbols = list(for2007['nyse'])
    n = len(symbols)
    prices_out = []
    for s in symbols:
        dataframe = pd.read_csv('GAN/Data/Stocks/%s.us.txt' % s, index_col=None, parse_dates=['Date'])
        prices = list(dataframe['Close'])[-500:] # trailing window 500 days
        prices_out.append(prices)
    return symbols, prices_out
names, prices = load_data()
n = len(names)

In [239]:
def assets_historical_covariances(prices):
    prices = matrix(prices)
    rows, cols = prices.shape
    returns = empty([rows, cols - 1])
    for r in range(rows):
        for c in range(cols - 1):
            p0, p1 = prices[r, c], prices[r, c + 1]
            returns[r, c] = (p1 / p0) - 1
    covars = cov(returns)
    covars = covars * 250  # Annualize covariances
    return covars

C = assets_historical_covariances(prices)

In [241]:
display(pandas.DataFrame(C, columns=names, index=names))

,amzn,abt,aes,ibm,ip,iff,amd,adbe,apd,agn,...,win,ash,rrd,aa,cce,spgi,xrx,hpq,lll,itt
amzn,0.067381,0.008414,0.009596,0.008184,0.007041,0.006864,0.028330,0.023390,0.005145,0.012181,...,0.020971,0.009675,0.012497,0.016401,0.010679,0.016158,0.009267,0.013993,0.005820,0.012117
abt,0.008414,0.040351,0.013389,0.013018,0.013983,0.013912,0.029427,0.017342,0.013412,0.018921,...,0.017436,0.014314,0.021267,0.021589,0.008246,0.020295,0.014111,0.019733,0.010285,0.018851
aes,0.009596,0.013389,0.066717,0.014489,0.024576,0.021880,0.030517,0.018755,0.016711,0.007896,...,0.027580,0.017841,0.025764,0.034182,0.012776,0.017638,0.020601,0.026892,0.009468,0.021972
ibm,0.008184,0.013018,0.014489,0.032515,0.013391,0.011211,0.026259,0.012417,0.011315,0.003529,...,0.015940,0.009859,0.016713,0.021482,0.007543,0.014193,0.016570,0.019437,0.007232,0.020756
ip,0.007041,0.013983,0.024576,0.013391,0.050567,0.018760,0.034282,0.014552,0.019724,0.010774,...,0.026127,0.019557,0.032588,0.034484,0.009632,0.019573,0.022057,0.025465,0.012422,0.034775
iff,0.006864,0.013912,0.021880,0.011211,0.018760,0.043193,0.027055,0.017253,0.017129,0.010557,...,0.010101,0.015578,0.018776,0.019758,0.011319,0.017212,0.014865,0.019457,0.010118,0.019555
amd,0.028330,0.029427,0.030517,0.026259,0.034282,0.027055,0.539790,0.037480,0.017076,0.024943,...,0.053039,0.032661,0.047644,0.070277,0.015184,0.032783,0.052901,0.054421,0.019076,0.043529
adbe,0.023390,0.017342,0.018755,0.012417,0.014552,0.017253,0.037480,0.053037,0.014719,0.013164,...,0.009478,0.015384,0.020341,0.021091,0.012707,0.020864,0.015357,0.023714,0.011569,0.018164
apd,0.005145,0.013412,0.016711,0.011315,0.019724,0.017129,0.017076,0.014719,0.030998,0.005614,...,0.016006,0.015230,0.020214,0.028343,0.009905,0.018924,0.016828,0.017096,0.010280,0.024800
agn,0.012181,0.018921,0.007896,0.003529,0.010774,0.010557,0.024943,0.013164,0.005614,0.079242,...,0.013963,0.004185,0.019393,0.012516,0.005143,0.012848,0.010609,0.012030,0.011901,0.011664
